## Init For Agents Testing

In [2]:
# Setup: Load environment variables and dependencies
import os
import sys
from pathlib import Path

from jinja2 import Environment, FileSystemLoader

project_root = Path.cwd()
src_path = project_root / "src"

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"✓ Added to sys.path: {src_path}")

from LabAgentSkill import skills_utils
from LabAgentSkill.SkillAwareAgent import SkillAwareAgent

root_dir = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
env_path = root_dir / ".env"
env = {}

if env_path.exists():
    for line in env_path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        key, value = line.split("=", 1)
        env[key.strip()] = value.strip()

# Set API key
os.environ["OPENAI_API_KEY"] = env.get("OPENAI_API_KEY", os.environ.get("OPENAI_API_KEY", ""))
env = Environment(loader=FileSystemLoader('prompts/'))  
skills_folder = Path("/home/snt/projects_lujun/LabAgentSkill/skillsHub/skills_finer")
all_skills = skills_utils.read_all_skills_metadata(skills_folder)
for skill in all_skills:
    print(f"  - {skill['name']}: {skill['description']}")

# model_name = "gpt-4o-mini"
# base_url = None
model_name = "google/gemma-3-270m-it"
# model_name = "Qwen/Qwen2.5-0.5B-Instruct"
base_url = "http://127.0.0.1:8001/v1"

  - XBRL-tag-classification: Classify financial text into specific XBRL tags by analyzing semantic cues, context, and category boundaries.
  - algorithmic-art: Creating algorithmic art using p5.js with seeded randomness and interactive parameter exploration. Use this when users request creating art using code, generative art, algorithmic art, flow fields, or particle systems. Create original algorithmic art rather than copying existing artists' work to avoid copyright violations.
  - brand-guidelines: Applies Anthropic's official brand colors and typography to any sort of artifact that may benefit from having Anthropic's look-and-feel. Use it when brand colors or style guidelines, visual formatting, or company design standards apply.
  - canvas-design: Create beautiful visual art in .png and .pdf documents using design philosophy. You should use this skill when the user asks to create a poster, piece of art, design, or other static piece. Create original visual designs, never copying e

## Load Data - Sentimental Analysis

In [17]:
from datasets import load_dataset
dataset_name = "Volavion/finer-139-xbrl-nonempty"
loaded_dataset = load_dataset(dataset_name, split="train")
loaded_df = loaded_dataset.to_pandas()
loaded_df = loaded_df.reset_index(drop=True)

In [18]:
import pandas as pd
random_selected_df = loaded_df.sample(n=250, random_state=42)
new_df = pd.DataFrame(columns=["sentence", "tag_token", "tag_name"])
for idx, row in random_selected_df.iterrows():
    sentence_list = row["tokens"]
    tag_indices = row["tag_indices"]
    tag_tokens = [sentence_list[i] for i in tag_indices]
    tag_names = row["tag_names"]
    for tag_token, tag_name in zip(tag_tokens, tag_names):
        print (f"  - {tag_token}: {tag_name}")
        sentence = " ".join(sentence_list)
        print (f"Sentence: {sentence}")
        new_row_dict = {
            "sentence": sentence,
            "tag_token": tag_token,
            "tag_name": tag_name
        }
        new_row_df = pd.DataFrame([new_row_dict])
        new_df = pd.concat([new_df, new_row_df], ignore_index=True)

new_df = new_df[new_df["tag_token"].str.replace(",", "").str.replace(".", "").str.replace("-", "").str.strip().str.isnumeric()].reset_index(drop=True)


  - two: B-NumberOfOperatingSegments
Sentence: Accordingly , CNX Resources is the sole sponsor of the Partnership , and we may refer to CNX Resources as the “ Sponsor ” throughout this Quarterly Report on Form 10-Q . Description of Business Our midstream assets consist of two operating segments that we refer to as our “ Anchor Systems ” and “ Additional Systems ” based on their relative current cash flows , growth profiles , capital expenditure requirements and the timing of their development .
  - 17.43: B-TreasuryStockAcquiredAverageCostPerShare
Sentence: During the nine months ended September 30 , 2019 , the Company repurchased 0.8 million shares of common stock under the Program at an average price per share of $ 17.43 for a total of $ 14.6 million , excluding commissions .
  - 14.6: B-TreasuryStockValueAcquiredCostMethod
Sentence: During the nine months ended September 30 , 2019 , the Company repurchased 0.8 million shares of common stock under the Program at an average price per 

In [21]:
from datasets import Dataset

ds = Dataset.from_pandas(new_df)
ds.push_to_hub("Volavion/finer-139-numeric-sampled", private=False)

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 419.26ba/s]
Processing Files (1 / 1): 100%|██████████| 40.5kB / 40.5kB, 50.6kB/s  
New Data Upload: 100%|██████████| 40.5kB / 40.5kB, 50.6kB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.45s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/Volavion/finer-139-numeric-sampled/commit/31d85461da7733a376ff6921ec11933eea5c331e', commit_message='Upload dataset', commit_description='', oid='31d85461da7733a376ff6921ec11933eea5c331e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Volavion/finer-139-numeric-sampled', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Volavion/finer-139-numeric-sampled'), pr_revision=None, pr_num=None)

## Select skills 

In [3]:
import time
import json
from datetime import datetime
import pandas as pd
from tqdm import tqdm

from LabAgentSkill.evaluate import get_predicted_label

print(f"Using model: {model_name}")
# Initialize agents
agent_skill_aware = SkillAwareAgent(use_chat_history=True, use_trim_messages=True, model=model_name, base_url=base_url)
agent_skill_exec_agent = SkillAwareAgent(use_chat_history=True, use_trim_messages=True, model=model_name, base_url=base_url)
agent_simple = SkillAwareAgent(use_chat_history=True, use_trim_messages=False, model=model_name, base_url=base_url)

p_exec_imdb_temp = env.get_template('p_exec_imdb.jinja')
p_skill_select_temp = env.get_template('p_skill_select.jinja')
p_skill_discov_temp = env.get_template('p_skill_discov.jinja')
p_skill_exec_temp = env.get_template('p_skill_exec.jinja')
p_default_system_temp = env.get_template('p_default_system.jinja')

# JSONL output path
output_dir = "/home/snt/projects_lujun/LabAgentSkill/assets/results/"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
jsonl_path = output_dir+f"imdb_300_standard_{model_name.split('/')[-1]}_{timestamp}.jsonl"
print(f"Results will be saved to: {jsonl_path}")

skill_count = 0
count_row = 0
if os.path.exists(jsonl_path):
    df_exist = pd.read_json(jsonl_path, lines=True)
    count_row = len(df_exist)
    print(f"Resume from row: {count_row}")


# Process each sample
for idx, row in tqdm( loaded_df.iterrows(), total=len(loaded_df), desc="Processing samples",):

    if idx < count_row:
        continue
    sample_start_time, text, true_label = time.time(), row.text, row.label_name

    # Step 1: Skill Selection
    print(f"Start Skill Selection Phase for Sample {idx + 1}/{len(loaded_df)}")
    skill_context = "\n".join([
        f"- **{skill['name']}**: {skill['description']}"
        for skill in all_skills
    ])

    p_skill_select = p_skill_select_temp.render(SKILL_CONTEXT=skill_context)
    p_exec_imdb = p_exec_imdb_temp.render(text=text)
    skill_select_resp = agent_skill_aware.chat(user_input=p_exec_imdb, custom_system_prompt=p_skill_select)
    selected_skills = skills_utils.parse_skills_from_json_response(json_response=skill_select_resp, skills_hub_dir=skills_folder)

    # Track whether "movie-sentiment-analysis" was selected in Step 1 
    selected_skill_names_step1 = [s["name"] for s in selected_skills]
    hit_target_skill = "movie-sentiment-analysis" in selected_skill_names_step1 ## This is hard Coded

    skill_execution_context = ""
    for skill_meta in selected_skills:
        skill_execution_context += (
            f"SKill {skill_count + 1}: \n"
            f"{skill_meta['description']}\n"
            f"{'\n'.join(skill_meta['body'].split('\n')[1:])}\n\n"
        )
        skill_count += 1

    skill_count_prev = skill_count

    # Step 2: Skill Discovery
    discovery_rounds = 0
    while len(selected_skills) > 0:
        p_skill_discov = p_skill_discov_temp.render(SKILL_CONTEXT=skill_execution_context)
        skill_discov_resp = agent_skill_exec_agent.chat(user_input=p_skill_discov, custom_system_prompt=p_default_system_temp.render())
        selected_skills = skills_utils.parse_skills_from_json_response(json_response=skill_discov_resp, skills_hub_dir=skills_folder)

        for skill_meta in selected_skills:
            skill_execution_context += (
                f"SKill {skill_count + 1}: \n"
                f"{skill_meta['description']}\n"
                f"{'\n'.join(skill_meta['body'].split('\n')[1:])}\n\n"
            )
            skill_count += 1
        discovery_rounds += 1
    new_skills_found = skill_count - skill_count_prev


    # print(f"End of skill discovery phase. Found total of new skills: {new_skills_found}")
    # Step 3: Query Execution
    p_skill_exec = p_skill_exec_temp.render(SKILL_CONTEXT=skill_execution_context)
    imdb_exec_response = agent_skill_exec_agent.chat(user_input=p_exec_imdb, custom_system_prompt=p_skill_exec)
    message_classification = skills_utils.parse_message_from_json_response(imdb_exec_response)
    is_correct = true_label.lower() in message_classification.strip().lower()

    predicted_label = get_predicted_label(message_classification)
    sample_end_time = time.time()
    sample_elapsed = sample_end_time - sample_start_time
    chat_history_agent_skill_select = agent_skill_aware.get_human_ai_message_history()
    chat_history_agent_exec = agent_skill_exec_agent.get_human_ai_message_history()

    # Build record and append to JSONL
    record = {
        "index": int(idx),
        "text": text,
        "true_label": true_label,
        "predicted_label": predicted_label,
        "raw_response": message_classification,
        "correct": is_correct,
        "selected_skills_step1": selected_skill_names_step1,
        "hit_target_skill": hit_target_skill,
        "new_skills_discovered": new_skills_found,
        "discovery_rounds": discovery_rounds,
        "elapsed_seconds": round(sample_elapsed, 4),
        "model": model_name,
        "timestamp": datetime.now().isoformat(),
        "chat_history_agent_skill_select": chat_history_agent_skill_select,
        "chat_history_agent_exec": chat_history_agent_exec,
        "task_type": "agent_skill_based"
        
    }

    dataframe_record = pd.DataFrame([record])
    dataframe_record.to_json(jsonl_path, orient="records", lines=True, mode="a" if os.path.exists(jsonl_path) else "w")
    agent_skill_aware.clear_history()
    agent_skill_exec_agent.clear_history()

    ######################################################################################################################################
    sample_start_time, text, true_label = time.time(), row.text, row.label_name

    p_exec_imdb = p_exec_imdb_temp.render(text=text)
    p_exec_imdb_sys = p_default_system_temp.render()
    imdb_exec_response = agent_simple.chat(user_input=p_exec_imdb, custom_system_prompt=p_exec_imdb_sys)
    message_classification = skills_utils.parse_message_from_json_response(imdb_exec_response)
    is_correct = true_label.lower() in message_classification.strip().lower()

    predicted_label = get_predicted_label(message_classification)
    sample_end_time = time.time()
    sample_elapsed = sample_end_time - sample_start_time
    chat_history_agent_exec = agent_simple.get_human_ai_message_history()

    # Build record and append to JSONL
    record = {
        "index": int(idx),
        "text": text,
        "true_label": true_label,
        "predicted_label": predicted_label,
        "raw_response": message_classification,
        "correct": is_correct,
        "selected_skills_step1": "",
        "hit_target_skill": "",
        "new_skills_discovered": "",
        "discovery_rounds": "",
        "elapsed_seconds": round(sample_elapsed, 4),
        "model": model_name,
        "timestamp": datetime.now().isoformat(),
        "chat_history_agent_skill_select": "",
        "chat_history_agent_exec": chat_history_agent_exec,
        "task_type": "agent_simple"
        
    }

    dataframe_record = pd.DataFrame([record])
    dataframe_record.to_json(jsonl_path, orient="records", lines=True, mode="a" if os.path.exists(jsonl_path) else "w")
    agent_simple.clear_history()

    ######################################################################################################################################
    sample_start_time, text, true_label = time.time(), row.text, row.label_name
    skill_context_all =  "The following are skills informaiton you can use as a reference for user request:\n".join([
        f"- **{skill['name']}**:\n {skill['description']} **:\n {skill['body']}"
        for skill in all_skills
    ])
    p_exec_imdb = p_exec_imdb_temp.render(text= text + skill_context_all)
    p_exec_imdb_sys = p_default_system_temp.render()

    imdb_exec_response = agent_simple.chat(user_input=p_exec_imdb, custom_system_prompt=p_exec_imdb_sys)
    message_classification = skills_utils.parse_message_from_json_response(imdb_exec_response)
    is_correct = true_label.lower() in message_classification.strip().lower()

    predicted_label = get_predicted_label(message_classification)
    sample_end_time = time.time()
    sample_elapsed = sample_end_time - sample_start_time
    chat_history_agent_exec = agent_simple.get_human_ai_message_history()

    # Build record and append to JSONL
    record = {
        "index": int(idx),
        "text": text,
        "true_label": true_label,
        "predicted_label": predicted_label,
        "raw_response": message_classification,
        "correct": is_correct,
        "selected_skills_step1": "",
        "hit_target_skill": "",
        "new_skills_discovered": "",
        "discovery_rounds": "",
        "elapsed_seconds": round(sample_elapsed, 4),
        "model": model_name,
        "timestamp": datetime.now().isoformat(),
        "chat_history_agent_skill_select": "",
        "chat_history_agent_exec": chat_history_agent_exec,
        "task_type": "agent_skill_full_context"
        
    }

    dataframe_record = pd.DataFrame([record])
    dataframe_record.to_json(jsonl_path, orient="records", lines=True, mode="a" if os.path.exists(jsonl_path) else "w")
    agent_simple.clear_history()

print(f"\n{'='*60}")
print(f"All {len(loaded_df)} samples processed. Results saved to: {jsonl_path}")
print(f"{'='*60}")

Using model: google/gemma-3-270m-it
✓ SkillAwareAgent initialized
  Model: google/gemma-3-270m-it
  Base URL: http://127.0.0.1:8001/v1
  Chat History: ENABLED ✓
  Trim Messages: ENABLED ✓
✓ SkillAwareAgent initialized
  Model: google/gemma-3-270m-it
  Base URL: http://127.0.0.1:8001/v1
  Chat History: ENABLED ✓
  Trim Messages: ENABLED ✓
✓ SkillAwareAgent initialized
  Model: google/gemma-3-270m-it
  Base URL: http://127.0.0.1:8001/v1
  Chat History: ENABLED ✓
  Trim Messages: DISABLED ✗
Results will be saved to: /home/snt/projects_lujun/LabAgentSkill/assets/results/imdb_300_standard_gemma-3-270m-it_20260211_181940.jsonl


Processing samples:   0%|          | 0/300 [00:00<?, ?it/s]

Start Skill Selection Phase for Sample 1/300
✓ Chat history cleared
✓ Chat history cleared
✓ Chat history cleared


Processing samples:   0%|          | 1/300 [00:00<03:01,  1.64it/s]

✓ Chat history cleared
Start Skill Selection Phase for Sample 2/300
✓ Chat history cleared
✓ Chat history cleared
✓ Chat history cleared


...

✓ Chat history cleared
Start Skill Selection Phase for Sample 299/300
✓ Chat history cleared
✓ Chat history cleared
✓ Chat history cleared


Processing samples: 100%|█████████▉| 299/300 [20:41<00:01,  1.27s/it]

✓ Chat history cleared
Start Skill Selection Phase for Sample 300/300
✓ Chat history cleared
✓ Chat history cleared
✓ Chat history cleared


Processing samples: 100%|██████████| 300/300 [20:42<00:00,  4.14s/it]

✓ Chat history cleared

All 300 samples processed. Results saved to: /home/snt/projects_lujun/LabAgentSkill/assets/results/imdb_300_standard_gemma-3-270m-it_20260211_181940.jsonl


In [13]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score  # F1 supports macro/weighted averages [web:2]

results_df = pd.read_json(jsonl_path, lines=True)

ESTIMATED_GPU_RAM_GB = 44.3

for task_type, g in results_df.groupby("task_type", dropna=False):

    # --- ACC / F1: exclude "unknown" predictions ---
    pred_lower = g["predicted_label"].astype(str).str.lower()
    valid_mask = pred_lower.ne("unknown")
    valid_df = g[valid_mask]
    unknown_count = int((~valid_mask).sum())

    if len(valid_df) > 0:
        y_true = valid_df["true_label"].astype(str).str.lower()
        y_pred = valid_df["predicted_label"].astype(str).str.lower()

        acc = accuracy_score(y_true, y_pred)
        f1_macro = f1_score(y_true, y_pred, average="macro", zero_division=0)      # macro avg [web:2]
        f1_weighted = f1_score(y_true, y_pred, average="weighted", zero_division=0)  # weighted avg [web:2]
        correct = int((y_true == y_pred).sum())
        denom = len(valid_df)
    else:
        acc = float("nan")
        f1_macro = float("nan")
        f1_weighted = float("nan")
        correct = 0
        denom = 0

    # --- Hit Rate: count non-empty hit_target_skill ---
    hit_count = int(g["hit_target_skill"].fillna("").astype(str).eq("True").sum())
    hit_rate = hit_count / len(g) if len(g) > 0 else float("nan")

    # --- VRAM-Hours (GB·h): VRAM(GB) * time(hours) ---
    total_time_sec = g["elapsed_seconds"].fillna(0).astype(float).sum()
    total_minutes = (total_time_sec / 60.0)
    total_vram_minutes = total_minutes * ESTIMATED_GPU_RAM_GB
    avg_minutes= total_minutes / len(g) if len(g) > 0 else float("nan")
    avg_vram_minutes = total_vram_minutes / len(g) if len(g) > 0 else float("nan")
    # --- Print only the requested metrics ---
    # print(f"[task_type={task_type}] "
    #       f"ACC={acc:.4f} ({correct}/{denom}, unknown_excluded={unknown_count}) | "
    #       f"F1_weighted={f1_weighted:.4f} | "
    #       f"HitRate={hit_rate:.4f} ({hit_count}/{len(g)}) | "
    #       f"AvgHours={avg_hours:.4f} | AvgVRAM-Hours={avg_vram_hours:.4f} GB·h"
    # )

    print (f"{task_type} {acc:.4f} {f1_weighted:.4f} {hit_rate:.4f} {avg_minutes:.4f} {avg_vram_minutes:.4f}")


agent_simple 0.6373 0.6093 0.0000 0.0024 0.1072
agent_skill_based 0.6667 0.6268 0.0000 0.0063 0.2813
agent_skill_full_context 0.5000 0.4176 0.0000 0.0600 2.6562


In [8]:
results_df.hit_target_skill.value_counts(dropna=False)

hit_target_skill
         600
False    300
Name: count, dtype: int64